In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import seaborn as sns

from scipy.stats import pearsonr

from sklearn.linear_model import ElasticNetCV, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, r2_score

In [2]:
import sys
import os
from importlib import reload

# Add the path of your package
package_path = os.path.abspath("/Genomics/pritykinlab/tamjeed/github_packages/GlossPath/")
sys.path.insert(0, package_path)

In [3]:
import matplotlib.pyplot as plt

In [4]:
import pickle

In [5]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/gut_data_aug_11_2024.h5ad')

In [6]:
from scipy.sparse import csr_matrix

In [7]:
with open('../pathway_computations/cellchatdb.pickle', 'rb') as handle:
    cellchatdb = pickle.load(handle)

In [12]:
with open('../pathway_computations/to_genesets_nn.pickle', 'rb') as handle:
    to_genesets = pickle.load(handle)

In [9]:
with open('../pathway_computations/combined_cellchatdb_nn_signaling.pickle', 'rb') as handle:
    combined_c_n = pickle.load(handle)

In [41]:
with open('../pathway_computations/combined_cellchatdb_nn_signaling_relaxed.pickle', 'rb') as handle:
    combined_c_n_relaxed = pickle.load(handle)

## creating 10 different multiple permutations

In [28]:
adata = og_adata.copy()
y = adata.obs['raw_biotin']
adata_df = adata.to_df().copy()
cd40_df = adata_df[adata_df.columns.intersection(cd40neighbors['cd40_and_neighbors'])]

np.random.seed(0)
random_factors = np.random.uniform(0.5, 1.5, size=(cd40_df.shape[0], 1))
random_factors.flatten()

result_df = np.round(cd40_df * random_factors).astype(int).astype(float)

adata_df[result_df.columns] = result_df

result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

print(np.percentile(result_y, 5))

adata.obs['biotin_raw_perturbed'] = result_y

csr_matrix(adata_df)
adata.X = csr_matrix(adata_df)

1.0


In [ ]:
adata = og_adata.copy()
adata_df = adata.to_df().copy()
pert_df = adata_df[adata_df.columns.intersection(combined_c_n['combined_c_n'])]

np.random.seed(i)
random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
random_factors.flatten()

result_df = np.round(pert_df * random_factors).astype(int).astype(float)

adata_df[result_df.columns] = result_df

result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

print(np.percentile(result_y, 5))

adata.obs['biotin_raw_perturbed'] = result_y

csr_matrix(adata_df)
adata.X = csr_matrix(adata_df)

In [11]:
y = og_adata.obs['raw_biotin']
for i in range(10):
    print(i)
    adata = og_adata.copy()
    adata_df = adata.to_df().copy()
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n['combined_c_n'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y

    csr_matrix(adata_df)
    adata.X = csr_matrix(adata_df)

    adata.write('../joint_notebooks/datasets/{}_gut_data_perturbed_combined_c_n_v2_oct_26_2024.h5ad'.format(i))

0
15.0
1
14.0
2
14.0
3
14.0
4
14.0
5
14.0
6
14.0
7
14.0
8
14.0
9
14.0


In [17]:
y = og_adata.obs['raw_biotin']
for i in range(10):
    print(i)
    adata = og_adata.copy()
    adata_df = adata.to_df().copy()
    pert_df = adata_df[adata_df.columns.intersection(to_genesets['to_Itgae'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y

    csr_matrix(adata_df)
    adata.X = csr_matrix(adata_df)

    adata.write('../joint_notebooks/datasets/{}_gut_data_perturbed_itgae_v2_oct_26_2024.h5ad'.format(i))

0
15.0
1
14.0
2
14.0
3
14.0
4
14.0
5
14.0
6
14.0
7
14.0
8
14.0
9
14.0


## now do the same for lcmv sys and lcmv ln

In [35]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/lcmv_sys_data_aug_11_2024.h5ad')

In [36]:
for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n['combined_c_n'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_sys_data_perturbed_combined_c_n_v2_oct_26_2024.h5ad'.format(i))

0
2.0
1
2.0
2
2.0
3
2.0
4
2.0
5
2.0
6
2.0
7
2.0
8
2.0
9
2.0


In [37]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/lcmv_ln_data_aug_11_2024.h5ad')

for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n['combined_c_n'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_ln_data_perturbed_combined_c_n_v2_oct_26_2024.h5ad'.format(i))

0
1.0
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0


## relaxed thresholds

In [39]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/gut_data_aug_11_2024.h5ad')

In [42]:
y = og_adata.obs['raw_biotin']
for i in range(10):
    print(i)
    adata = og_adata.copy()
    adata_df = adata.to_df().copy()
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_5'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y

    csr_matrix(adata_df)
    adata.X = csr_matrix(adata_df)

    adata.write('../joint_notebooks/datasets/{}_gut_data_perturbed_combined_c_n_more_relaxed_5_v2_oct_26_2024.h5ad'.format(i))

0
15.0
1
14.0
2
14.0
3
14.0
4
14.0
5
14.0
6
14.0
7
14.0
8
14.0
9
14.0


In [43]:
y = og_adata.obs['raw_biotin']
for i in range(10):
    print(i)
    adata = og_adata.copy()
    adata_df = adata.to_df().copy()
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_6'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y

    csr_matrix(adata_df)
    adata.X = csr_matrix(adata_df)

    adata.write('../joint_notebooks/datasets/{}_gut_data_perturbed_combined_c_n_relaxed_6_v2_oct_26_2024.h5ad'.format(i))

0
15.0
1
14.0
2
14.0
3
14.0
4
14.0
5
14.0
6
14.0
7
14.0
8
14.0
9
14.0


In [44]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/lcmv_sys_data_aug_11_2024.h5ad')

In [45]:
for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_5'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_sys_data_perturbed_combined_c_n_more_relaxed_5_v2_oct_26_2024.h5ad'.format(i))

0
2.0
1
2.0
2
2.0
3
2.0
4
2.0
5
2.0
6
2.0
7
2.0
8
2.0
9
2.0


In [46]:
for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_6'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_sys_data_perturbed_combined_c_n_relaxed_6_v2_oct_26_2024.h5ad'.format(i))

0
2.0
1
2.0
2
2.0
3
2.0
4
2.0
5
2.0
6
2.0
7
2.0
8
2.0
9
2.0


In [47]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/lcmv_ln_data_aug_11_2024.h5ad')

In [48]:
for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_5'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_ln_data_perturbed_combined_c_n_more_relaxed_5_v2_oct_26_2024.h5ad'.format(i))

0
1.0
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0


In [49]:
for i in range(10):
    print(i)
    all_adata = og_adata.copy()
    all_adata_df = all_adata.to_df().copy()
    all_y = og_adata.obs['biotin_interaction'].copy()
    adata = all_adata[~all_adata.obs['donor']].copy()
    adata_df = adata.to_df().copy()
    y = all_y[adata.obs.index]
    
    pert_df = adata_df[adata_df.columns.intersection(combined_c_n_relaxed['combined_c_n_6'])]

    np.random.seed(i)
    random_factors = np.random.uniform(0.5, 1.5, size=(pert_df.shape[0], 1))
    random_factors.flatten()

    result_df = np.round(pert_df * random_factors).astype(int).astype(float)

    adata_df[result_df.columns] = result_df

    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)

    print(np.percentile(result_y, 5))

    adata.obs['biotin_raw_perturbed'] = result_y
    
    all_adata_df.loc[adata_df.index, adata_df.columns] = adata_df.copy()
    
    all_y[adata.obs.index] = result_y.copy()
    
    all_adata.obs['biotin_raw_perturbed'] = all_y.copy()
    all_adata.X = csr_matrix(all_adata_df)
    
    all_adata.write('../joint_notebooks/datasets/{}_lcmv_ln_data_perturbed_combined_c_n_relaxed_6_v2_oct_26_2024.h5ad'.format(i))

0
1.0
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0
